# Week 4: ETL Pipelines in Azure Databricks

In [53]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("DeltaLakeETL") \
    .getOrCreate()

In [ ]:
# Create a DataFrame for customer data
customer_data = [
    (2001, 'Shruti', 'India'),
    (2002, 'Priyanshi', 'Canada'),
    (2003, 'Prakhar', 'New York'),
    (2004, 'Shreya', 'USA'),
    (2005, 'Divya', 'Australia')
]

customer_columns = ["customer_id", "customer_name", "location"]
customer_df = spark.createDataFrame(customer_data, customer_columns)

# Write the DataFrame to Delta format
customer_df.write.format("delta").mode("overwrite").save("/content/delta/customer_dim")

# Create a DataFrame for order data
order_data = [
    (1001, 1, 2001, 1, 139.99, '2023-09-27 12:30:00'),
    (1002, 2, 2002, 2, 934.99, '2023-09-27 14:45:00'),
    (1003, 3, 2003, 1, 543.99, '2023-09-27 15:30:00'),
    (1004, 1, 2004, 2, 299.99, '2023-09-27 16:00:00'),
    (1005, 2, 2005, 1, 499.99, '2023-09-27 17:30:00')
]

order_columns = ["order_id", "product_id", "customer_id", "quantity", "order_amount", "order_date"]
order_df = spark.createDataFrame(order_data, order_columns)

# Write the order DataFrame to Delta format
order_df.write.format("delta").mode("overwrite").save("/content/delta/order_fact")

import dlt

@dlt.table
def order_fact():
    return spark.read.format("delta").load("/content/delta/order_fact")

@dlt.table
def customer_dim():
    return spark.read.format("delta").load("/content/delta/customer_dim")

# Show data from Delta Live Tables
df_dltCustomers = spark.read.format("delta").load("/content/delta/customer_dim")
df_dltCustomers.show()

df_dlt_Order_fact = spark.read.format("delta").load("/content/delta/order_fact")
df_dlt_Order_fact.show()
